In [5]:
# Gaussian Function over points in two dimensions
# Given mean in two dimensions, variance in two dimensions
import tensorflow as tf
def gaussian(x1, x2, mean1, mean2, var1, var2, rho):
    x_mean1 = tf.subtract(x1, mean1)
    x_mean2 = tf.subtract(x2, mean2)
    x_norm1 = tf.div(x_mean1, var1)
    x_norm2 = tf.div(x_mean2, var2)
    Z = tf.square(x_norm1) + tf.square(x_norm2) - 2*tf.div(tf.multiply(rho, tf.multiply(x_mu1, x_mu2)), tf.multiply(s1, s2))
    rho_square_term = 1-tf.square(rho)
    power_e = tf.exp(tf.div(-Z,2*rho_square_term))
    regularize_term = 2*np.pi*tf.multiply(tf.multiply(s1, s2), tf.sqrt(rho_square_term))
    gaussian = tf.div(power_e, regularize_term)
    return gaussian


    

In [7]:
# Number of parameters per gaussian
# 6*(Number of mixtures) parameters plus end of stroke. Total = 7
num_mixtures = 20
rnn_size = 400
n_out = 1 + 6*num_mixtures
with tf.variable_scope('mdn_dense'):
    w = tf.get_variable("out_w", [rnn_size, n_out], initializer = tf.contrib.layers.xavier_initializer(seed=0))
    b = tf.get_variable("out_b", [n_out, 1], initializer = tf.contrib.layers.xavier_initializer(seed=0))

# Feeding output from all 3 hidden lstm layers in MDN    
output = tf.nn.xw_plus_b(output_from_lstm_layers, mdn_w, mdn_b) #data flows through dense nn
    


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.


NameError: name 'output_from_lstm_layers' is not defined

In [8]:
def get_loss(pi, x1_data, x2_data, mean1, mean2, var1, var2, eos_data, eos, rho):
    gaussian = gaussian(x1_data, x2_data, mean1, mean2, var1, var2, rho)
    term1 = tf.multiply(gaussian, pi)
    # Do summation according to the formula
    summation = tf.reduce_sum(term1, 1, keep_dims = true)
    logloss_first = -tf.log(summation, 1e-20) # If the summation is by chance 0, we do not want log to go to infinity
    
    term2 = tf.multiply(eos, eos_data) + tf.multiply(1-eos, 1-eos_data) #modified Bernoulli -> eos probability
    logloss_second = -tf.log(term2)
    
    return tf.reduce_sum(term1 + term2) #do outer summation

# Models coefficients for MDN as a function of input from LSTM cells.
def get_mdn_coefficients(Z):
    eos_hat = Z[:][0:1] # end of sentence tokens
    # Split Z into slices of 6x1, for defining MDN coefficients for 2D gaussian.
    pi_hat, mean1_hat, mean2_hat, var1_hat, var2_hat, rho_hat = tf.split(Z[:, 1:], 6, 1)
    eos = tf.sigmoid(-1*eos_hat) # technically we gained a negative sign
    pi = tf.nn.softmax(pi_hat) # softmax z_pi:
    mu1 = mu1_hat; mu2 = mu2_hat # leave mu1, mu2 as they are
    sigma1 = tf.exp(sigma1_hat); sigma2 = tf.exp(sigma2_hat) # exp for sigmas
    rho = tf.tanh(rho_hat) # tanh for rho (squish between -1 and 1)
    return [eos, pi, mu1, mu2, sigma1, sigma2, rho]
    

[self.eos, self.pi, self.mu1, self.mu2, self.sigma1, self.sigma2, self.rho] = get_mdn_coefficients(output)   
loss = get_loss(pi, )